In [2]:
#required imports
import numpy as np
from scipy import optimize
import matplotlib.pyplot as plt

In [3]:
#open data file and read data
with open("ASTR19_S22_group_project_data.txt", encoding="utf-8") as file:
    raw_data = file.read()

#close file when done
file.close()

In [4]:
#init array to store each line
lineArray = []
#init place for the popped values to go that is not terminal output
junk = ""

#split raw data into lines, store in prepared array
lineArray = str.split(raw_data, "\n")

#remove empty strings from data
for i in lineArray:
    if i == "":
        lineArray.remove(i)

#remove header information in first three lines
junk = lineArray.pop(0)
junk = lineArray.pop(0)
junk = lineArray.pop(0)

In [5]:
#init arrays to hold components of the data
dayArray = []
timeArray = []
rawDataArray = []
hold = []

#split data components and place in arrays
for i in lineArray:
    hold = str.split(i, " ")
    #this split method is not ideal, but it's been ages and all the stuff I remember is in Java.
    #as such, this loop checks for blank strings and removes them.
    for j in hold:
        if j == '':
            hold.remove('')
    #still in the for loop, add each separated data component to the discrete arrays 
    dayArray.append(hold[0])
    timeArray.append(hold[1])
    rawDataArray.append(hold[2])

In [28]:
#initialize array to hold time data, and one to hold the data that has been cast
decimalDayArray = []
dataArray = []
#init a holder and counter
temp = 0

#store time as float, with minutes and hours represented as decimal portions of days
#cast each int day to float
for i in dayArray:
    decimalDayArray.append(float(i))

#reset for use as counter
temp = 0

for i in timeArray:
    hold = str.split(i, ":")
    decimalTemp = (float(hold[0]) / 24) + ((float(hold[1]) / 24) / 60)
    decimalDayArray[temp] = decimalDayArray[temp] + decimalTemp
    temp = temp + 1

#cast rawDataArray strings to float into new array
for i in rawDataArray:
    dataArray.append(float(i))

[1.3381944444444445, 1.6506944444444445, 2.3715277777777777, 2.6840277777777777, 3.40625, 3.7180555555555554, 4.440972222222222, 4.75, 5.477777777777778, 5.781944444444444, 6.5152777777777775, 6.813194444444444, 7.558333333333334, 7.844444444444444, 8.140277777777778, 8.875694444444445, 9.171527777777778, 9.906944444444445, 10.20138888888889, 10.498611111111112, 11.229861111111111, 11.53888888888889, 12.255555555555556, 12.572916666666666, 13.282638888888888, 13.600694444444445, 14.306944444444444, 14.625, 15.332638888888889, 15.64861111111111, 16.356944444444444, 16.67222222222222, 17.381944444444443, 17.695138888888888, 18.40625, 18.716666666666665, 19.43125, 19.739583333333332, 20.45625, 20.760416666666668, 21.48472222222222, 21.783333333333335, 22.51736111111111, 22.805555555555557, 23.09861111111111, 23.82777777777778, 24.12361111111111, 24.854166666666668, 25.150694444444444, 25.447222222222223, 26.18263888888889, 26.49652777777778, 27.21875, 27.539583333333333, 28.25625, 28.5763

In [ ]:
#plot the data by itself w/ error
#from assignment: "assuming the root mean squared experimental error on the height of the tide is 0.25 ft"
f, ax = plt.subplots()
ax.errorbar(decimalDayArray, dataArray, yerr=.25, fmt='o')
ax.set_xlabel('Time(days)')
ax.set_ylabel('Tide Height(ft)')

In [30]:
#finding line of best fit
#a = 3.4
#b = 2.1
#c = 2
sig = 0.6
y_err = np.full(85,sig)

def f_line(x, a, b, c):
    return a * np.sin(b * x + c)

params, params_cov = optimize.curve_fit(f_line, decimalDayArray, dataArray)

a_fit = params[0]
b_fit = params[1]
c_fit = params[2]

#print(a_fit,b_fit,c_fit)
#y_fit = a_fit * np.sin(b_fit * decimalDayArray + c_fit)

y_fit = []
temp = 0

for i in decimalDayArray:
    y_fit.append(a_fit * np.sin(b_fit * i) + c_fit)
    
ax.plot(decimalDayArray, y_fit)
f.set_facecolor("white")

f.savefig("test output.png", bbox_inches = "tight", dpi = 300)

In [ ]:
#replot as histogram
axHist = plt.subplot()

axHist.hist(dataArray, 20)
axHist.set_xlabel("Height (ft)")
axHist.set_ylabel("Frequency of occurrence")

